<center><h1 style="font-size:40px;">Exercise II: Classification</h1></center>

---

For this exercise you are given two classification problems with a fixed training-, validation- and test dataset. A spiral dataset and a japanese vowels dataset.

The **task** is to do model selection, coming up with your optimal MLP architecture together with the hyperparameters. For this part no code for the configuration or training are given. Please implement this based on the previous parts. A small evaluation section is provided to observe the confusion matrix for the prediction vs target.

# Data
## Japanese vowels dataset
This data set is taken from the UCI Machine Learning Repository [https://archive.ics.uci.edu/ml/datasets/Japanese+Vowels]. In short, nine male speakers uttered two Japanese vowels /ae/ successively. For each utterance, a discrete times series was produced where each time point consists of 12 (LPC cepstrum) coefficients. The length of each time series was between 7-29. 
Here we treat each point of the time series as a feature (12 inputs). In total we have 9961
data points which then has been divided into 4274 for training, 2275 for validation and 3412 for test. The original data files are provided as *ae.train* and *ae.test*. The task is now based on a single sample value of one of the speakers, determine which speaker it was. This is, in summary, a 9-class classification problem with 12 input values for each case.

## Spiral data
This is the "famous" spiral dataset that consists of two 2-D spirals, one for each class. The perfect classification boundary is also a spiral. The cell "PlotData" will plot this dataset.

# Code

The following code allows us to edit imported files without restarting the kernel for the notebook

In [1]:
%load_ext autoreload
%autoreload 2

# Hacky solution to access the global utils package
import sys,os
sys.path.append(os.path.dirname(os.path.realpath('')))

In [24]:

import numpy as np
import matplotlib.pyplot as plt
import torch
import pytorch_lightning as pl

from config import LabConfig
from dataset import MLPData
from utils.model import Model
from utils.progressbar import LitProgressBar
from utils.model import Model
from torch.utils.data import TensorDataset, DataLoader
import torchmetrics
from utils import (
    plot,
    progressbar
) 

In [25]:
cfg = LabConfig()

## Task 1
**TODO:** Present an MLP for the Japanese vowels dataset with associated hyperparameters that maximizes the validation performance and give the test performance you obtained. \
**TODO:** Motivate the choice of parameters and implementation. 

**Hint 2:** This problem is a 9-class classification problem, meaning that you should use a specific output activation function (*out_act_fun*) and a specific loss/error function (*cost_fun*).

In [47]:
# Load dataset - Vowels
from utils.utils import onehot2int
def numpy2Dataloader(x,y, batch_size=50, num_workers=10,**kwargs):
    return DataLoader(
        TensorDataset(
            torch.from_numpy(x).float(), 
            torch.from_numpy(np.argmax(y,axis=1)).long()
        ),
        batch_size=batch_size,
        num_workers=num_workers,
        **kwargs
    )

x_train, y_train, x_val, y_val, x_test, y_test = MLPData.vowels(file_name_train=cfg.ae_train, file_name_test=cfg.ae_test)

train_loader = numpy2Dataloader(x_train,y_train)
val_loader =  numpy2Dataloader(x_val,y_val)
test_loader =  numpy2Dataloader(x_test,y_test)

num_classes = 9
print(f'|{"Type":10} | {"Input size":10} | {"Target size":10}|')
print(f'|{"-"*11}|{"-"*12}|{"-"*12}|')
print(f'|{"train":10} | {str(x_train.shape):10} | {str(y_train.shape):10} |')
print(f'|{"val":8}   | {str(x_val.shape):10} | {str(y_val.shape):10} |')
print(f'|{"test":9}  | {str(x_test.shape):10} | {str(y_test.shape):10} |')

|Type       | Input size | Target size|
|-----------|------------|------------|
|train      | (4274, 12) | (4274, 9)  |
|val        | (2275, 12) | (2275, 9)  |
|test       | (3412, 12) | (3412, 9)  |


In [48]:
# TODO - Create model 
class MLP(torch.nn.Module):
    def __init__(self, 
                inp_dim=None,         
                hidden_nodes=1, # number of nodes in hidden layer
                num_out=None,
                **kwargs
            ):
        super(MLP, self).__init__()
        self.fc1 = torch.nn.Linear(inp_dim, hidden_nodes)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_nodes, num_out)


    def forward(self, x):
        hidden = self.fc1(x)
        relu = self.relu(hidden)
        output = self.fc2(relu)
        return torch.sigmoid(output)                       

In [53]:
# TODO - Setup configurations
config = {
    'max_epochs':5,
    'model_params':{
        'inp_dim':x_train.shape[1],         
        'hidden_nodes':10,   # activation functions for the hidden layer
        'num_out':9 # if binary --> 1 |  regression--> num inputs | multi-class--> num of classes
    },
    'criterion':torch.nn.CrossEntropyLoss(), # error function
    'optimizer':{
        "type":torch.optim.Adam,
        "args":{
            "lr":0.01,
        }
    }
}

In [54]:
# TODO - Run model
model = Model(MLP(**config["model_params"]),**config)

trainer = pl.Trainer(
            max_epochs=config['max_epochs'], 
            gpus=cfg.GPU,
            logger=pl.loggers.TensorBoardLogger(save_dir=cfg.TENSORBORD_DIR),
            callbacks=[LitProgressBar()],
            progress_bar_refresh_rate=1,
            weights_summary='full', # Can be None, top or full
            num_sanity_val_steps=10,   
        )
trainer.fit(
    model, 
    train_dataloader=train_loader,
    val_dataloaders=val_loader
);


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name       | Type             | Params
------------------------------------------------
0 | model      | MLP              | 229   
1 | model.fc1  | Linear           | 130   
2 | model.relu | ReLU             | 0     
3 | model.fc2  | Linear           | 99    
4 | criterion  | CrossEntropyLoss | 0     
------------------------------------------------
229       Trainable params
0         Non-trainable params
229       Total params
0.001     Total estimated model params size (MB)


In [55]:
# TODO - Validate performance
plot.stats_class(x_train, y_train, 'Training', model)
plot.stats_class(x_val, y_val, 'Validation', model)



 ########## STATISTICS for Training Data ########## 

Accuracy   7.5
Sensitivity   0.4822180627047262
Specificity   0.8772227421619092

 ##################################################

 ########## STATISTICS for Validation Data ########## 

Accuracy   7.381978021978022
Sensitivity   0.3982417582417582
Specificity   0.8729670329670329

 ##################################################


## Task 2
For this last exercise the task is to train a binary classifier for the spiral problem. The aim is to get *zero* classification error for the training data (there is no test data) with as small as possible model, in terms of the number of trainable weights. Also plot the boundary to see if it resembles a spriral. To pass this question you should at least try!

**TODO:** Train a classifier for the spiral problem with the aim of zero classification error with as small as possible model. 

**TODO:** Motivate the choice of parameters and implementation.

In [188]:
# Load dataset
def numpy2Dataloader(x,y, batch_size=50, num_workers=10,**kwargs):
    return DataLoader(
        TensorDataset(
            torch.from_numpy(x).float(), 
            torch.from_numpy(y).float().unsqueeze(-1)
        ),
        batch_size=batch_size,
        num_workers=num_workers,
        **kwargs
    )
x_train, y_train = MLPData.spiral(cfg.spiral_path)
train_loader = numpy2Dataloader(x_train,y_train)

In [189]:
print(f'|{"Type":10} | {"Input size":10} | {"Target size":10}|')
print(f'|{"-"*11}|{"-"*12}|{"-"*12}|')
print(f'|{"train":10} | {str(x_train.shape):10} | {str(y_train.shape):10} |')
print(f'|{"val":8}   | {str(x_val.shape):10} | {str(y_val.shape):10} |')
print(f'|{"test":9}  | {str(x_test.shape):10} | {str(y_test.shape):10} |')

|Type       | Input size | Target size|
|-----------|------------|------------|
|train      | (194, 2)   | (194,)     |
|val        | (2275, 12) | (2275, 9)  |
|test       | (3412, 12) | (3412, 9)  |


## Implement the TODO's

In [190]:
# TODO - Create model 
class BinMLP(torch.nn.Module):
    def __init__(self, 
                inp_dim=None,         
                hidden_nodes=1, # number of nodes in hidden layer
                num_out=None,
                **kwargs
            ):
        super(BinMLP, self).__init__()
        self.fc1 = torch.nn.Linear(inp_dim, hidden_nodes)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_nodes, num_out)


    def forward(self, x):
        hidden = self.fc1(x)
        relu = self.relu(hidden)
        output = self.fc2(relu)
        return torch.sigmoid(output)  

In [195]:
# TODO - Setup configurations
config = {
    'max_epochs':10,
    'model_params':{
        'inp_dim':x_train.shape[1],         
        'hidden_nodes':2,   # activation functions for the hidden layer
        'num_out':1 # if binary --> 1 |  regression--> num inputs | multi-class--> num of classes
    },
    'criterion':torch.nn.BCELoss(), # error function
    'optimizer':{
        "type":torch.optim.Adam,
        "args":{
            "lr":0.01,
        }
    }
}

In [196]:
# TODO - Run model
model = Model(BinMLP(**config["model_params"]),**config)

trainer = pl.Trainer(
            max_epochs=config['max_epochs'], 
            gpus=cfg.GPU,
            logger=pl.loggers.TensorBoardLogger(save_dir=cfg.TENSORBORD_DIR),
            callbacks=[LitProgressBar()],
            progress_bar_refresh_rate=1,
            weights_summary="top", # Can be None, top or full
            num_sanity_val_steps=10,   
        )
trainer.fit(
    model, 
    train_dataloader=train_loader,
    #val_dataloaders=val_loader
);

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name      | Type    | Params
--------------------------------------
0 | model     | BinMLP  | 9     
1 | criterion | BCELoss | 0     
--------------------------------------
9         Trainable params
0         Non-trainable params
9         Total params
0.000     Total estimated model params size (MB)


In [197]:
# TODO - Validate performance
plot.stats_class(x_train, y_train, 'Training', model)
#plot.stats_class(x_val, y_val, 'Validation', model)


 ########## STATISTICS for Training Data ########## 

Accuracy   0.520618556701031
Sensitivity   0.6701030927835051
Specificity   0.3711340206185567

 ##################################################


## Example of evaluation
Run the testset and evaluate the performance of the model.

In [198]:
# Move to correct device!
trainer.test(model, test_dataloaders=test_loader)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (50x12 and 2x2)

In [185]:
trainer.logged_metrics

{}

### Confusion matrix 

In [187]:
predictions = torch.argmax(torch.nn.functional.softmax(model(torch.from_numpy(x_train).float())),dim=1)
target = torch.argmax(torch.from_numpy(y_train), axis=1)

confuTst = torchmetrics.functional.confusion_matrix(predictions.detach().cpu(),target.int().detach().cpu(), cfg.AE_NUM_CLASSES)

plot.confusion_matrix(cm = confuTst.numpy(), 
                      normalize = False,
                      target_names = cfg.AE_CLASSES,
                      title = "Confusion Matrix: Test data")

C:\Users\moa_a\AppData\Local\Temp/ipykernel_2384/975203574.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = torch.argmax(torch.nn.functional.softmax(model(torch.from_numpy(x_train).float())),dim=1)


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)